<a href="https://colab.research.google.com/github/kareemullah123456789/1112232/blob/master/Section_4_Spark_DataFrames_and_Transformations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning PySpark
### Video series

### Packt Publishing

**Author**: Tomasz Drabas
**Date**:   2018-01-30





# Section 4: Spark DataFrames & Transformations

In this section we will look at the Spark DataFrames and the transformations available.

In [1]:
!pip install pyspark --quiet
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from pyspark import SparkContext
import numpy as np

# Initialize Spark context
sc = SparkContext("local", "section4")

## Creating DataFrames
### From RDDs

In [3]:
simple_rdd = sc.parallelize([
      ['2017-02-01','Rachel', 19, 156, 'Sydney']
    , ['2018-01-01','Albert',  3,  45, 'New York']
    , ['2018-03-02','Jack',   61, 190, 'Krakow']
    , ['2017-12-31','Skye',    8,  82, 'Harbin']
])

In [5]:
# Import the required classes
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.appName("ExampleApp").getOrCreate()

In [6]:
simple_df = spark.createDataFrame(
    simple_rdd,
    ['Date','Name', 'Age', 'Weight', 'Location']
)

In [7]:
simple_df.show()

+----------+------+---+------+--------+
|      Date|  Name|Age|Weight|Location|
+----------+------+---+------+--------+
|2017-02-01|Rachel| 19|   156|  Sydney|
|2018-01-01|Albert|  3|    45|New York|
|2018-03-02|  Jack| 61|   190|  Krakow|
|2017-12-31|  Skye|  8|    82|  Harbin|
+----------+------+---+------+--------+



### From JSON string

In [8]:
json_string = [
    '{"Date":"2017-02-01","Name":"Rachel","Age":19,"Weight":156,"Location":"Sydney"}',
    '{"Date":"2018-01-01","Name":"Albert","Age":3 ,"Weight":45 ,"Location":"New York"}',
    '{"Date":"2018-03-02","Name":"Jack"  ,"Age":61,"Weight":190,"Location":"Krakow"}',
    '{"Date":"2017-12-31","Name":"Skye"  ,"Age":8 ,"Weight":82 ,"Location":"Harbin"}'
]

simple_df_json = spark.read.json(sc.parallelize(json_string))
simple_df_json.show()

+---+----------+--------+------+------+
|Age|      Date|Location|  Name|Weight|
+---+----------+--------+------+------+
| 19|2017-02-01|  Sydney|Rachel|   156|
|  3|2018-01-01|New York|Albert|    45|
| 61|2018-03-02|  Krakow|  Jack|   190|
|  8|2017-12-31|  Harbin|  Skye|    82|
+---+----------+--------+------+------+



In [10]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


### Reading data

In [11]:
sample_df = spark.read.csv(
    '/content/drive/MyDrive/learningsparkdata/sample_data.csv'
    , header=True
)

In [12]:
sample_df.show(4)

+----------+-------+-------+------+-----+--------+------+
| OrderDate| Region|    Rep|  Item|Units|UnitCost| Total|
+----------+-------+-------+------+-----+--------+------+
|    1/6/16|   East|  Jones|Pencil|   95|    1.99|189.05|
|2017-03-02|Central| Kivell|Binder|   50|   19.99| 999.5|
|    2/9/16|Central|Jardine|Pencil|   36|    4.99|179.64|
|   2/26/16|Central|   Gill|   Pen|   27|   19.99|539.73|
+----------+-------+-------+------+-----+--------+------+
only showing top 4 rows



## Spark DataFrame schema
### RDDs reflection

In [13]:
simple_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Weight: long (nullable = true)
 |-- Location: string (nullable = true)



### Programmatically specifying schema

In [14]:
import pyspark.sql.types as typ
import datetime as dt

schema = [
      ('Date', typ.DateType())
    , ('Name', typ.StringType())
    , ('Age',  typ.IntegerType())
    , ('Weight', typ.IntegerType())
    , ('Location', typ.StringType())
]

schema = typ.StructType([typ.StructField(e[0], e[1], True) for e in schema])

simple_df_schema = spark.createDataFrame(
      simple_rdd
        .map(lambda row:
             [dt.datetime.strptime(row[0], '%Y-%m-%d')] + row[1:]
            )
    , schema=schema
)

simple_df_schema.show()

+----------+------+---+------+--------+
|      Date|  Name|Age|Weight|Location|
+----------+------+---+------+--------+
|2017-02-01|Rachel| 19|   156|  Sydney|
|2018-01-01|Albert|  3|    45|New York|
|2018-03-02|  Jack| 61|   190|  Krakow|
|2017-12-31|  Skye|  8|    82|  Harbin|
+----------+------+---+------+--------+



In [15]:
simple_df_schema.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Location: string (nullable = true)



### Automatically inferring schema while reading data

In [16]:
sample_df.printSchema()

root
 |-- OrderDate: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Rep: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Units: string (nullable = true)
 |-- UnitCost: string (nullable = true)
 |-- Total: string (nullable = true)



In [17]:
sample_df_inferred = spark.read.csv(
    '/content/drive/MyDrive/learningsparkdata/sample_data.csv'
    , header=True
    , inferSchema = True
)

In [18]:
sample_df_inferred.printSchema()

root
 |-- OrderDate: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Rep: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Units: integer (nullable = true)
 |-- UnitCost: double (nullable = true)
 |-- Total: double (nullable = true)



In [19]:
import pyspark.sql.functions as f

sample_df_inferred = (
    sample_df_inferred
    .withColumn('OrderDate'
                , f.to_date('OrderDate', 'MM/dd/yy')
               )
)

sample_df_inferred.show(4)

Py4JJavaError: An error occurred while calling o100.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 9) (92a28b22707a executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '1/6/16' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48)
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218)
	at org.apache.spark.sql

## .agg(...)

In [20]:
sample_df_inferred.agg(
    {
          'Total': 'avg'
    }
).show()

+------------------+
|        avg(Total)|
+------------------+
|456.46232558139553|
+------------------+



In [21]:
aggregations = [
      ('Total', f.min,    'Total_min')
    , ('Total', f.max,    'Total_max')
    , ('Total', f.avg,    'Total_avg')
    , ('Total', f.stddev, 'Total_stddev')
]

(
    sample_df_inferred
    .agg(*[e[1](e[0]).alias(e[2]) for e in aggregations])
    .show()
)

+---------+---------+------------------+-----------------+
|Total_min|Total_max|         Total_avg|     Total_stddev|
+---------+---------+------------------+-----------------+
|     9.03|  1879.06|456.46232558139553|447.0221038416717|
+---------+---------+------------------+-----------------+



## .sql(...)

In [22]:
sample_df_inferred.createOrReplaceTempView('sample_df_inferred')

(
    spark
    .sql('''
        SELECT
              MIN(Total)    AS Total_min
            , MAX(Total)    AS Total_max
            , AVG(Total)    AS Total_avg
            , STDDEV(Total) AS Total_std
        FROM sample_df_inferred
    ''')
    .show()
)

+---------+---------+------------------+-----------------+
|Total_min|Total_max|         Total_avg|        Total_std|
+---------+---------+------------------+-----------------+
|     9.03|  1879.06|456.46232558139553|447.0221038416717|
+---------+---------+------------------+-----------------+



In [23]:
(
    sample_df_inferred
    .selectExpr(
          'MIN(Total) AS Total_min'
        , 'MAX(Total) AS Total_max'
    )
).show()

+---------+---------+
|Total_min|Total_max|
+---------+---------+
|     9.03|  1879.06|
+---------+---------+



## Creating temporary views

In [24]:
sample_df_inferred.createTempView('sample_df_inferred')

AnalysisException: [TEMP_TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create the temporary view `sample_df_inferred` because it already exists.
Choose a different name, drop or replace the existing view,  or add the IF NOT EXISTS clause to tolerate pre-existing views.

In [ ]:
sample_df_inferred.createOrReplaceTempView('sample_df_inferred')

## Joining two DataFrames

In [25]:
regions = spark.createDataFrame(
    sc.parallelize([
        ('Central', 'Chicago')
        , ('West', 'Seattle')
        , ('East', 'Boston')
    ]),
    ['Region', 'Headquarters']
)

In [26]:
(
    sample_df_inferred.join(
        regions
        , on=['Region']
        , how='left_outer'
    )
    .orderBy('OrderDate')
    .show(4)
)

Py4JJavaError: An error occurred while calling o173.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 23.0 failed 1 times, most recent failure: Lost task 0.0 in stage 23.0 (TID 19) (92a28b22707a executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '1/6/16' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48)
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218)
	at org.apache.spark.

## Descriptive statistics

In [27]:
sample_df_inferred.describe().show()

+-------+-------+--------+------+------------------+------------------+------------------+
|summary| Region|     Rep|  Item|             Units|          UnitCost|             Total|
+-------+-------+--------+------+------------------+------------------+------------------+
|  count|     43|      43|    43|                43|                43|                43|
|   mean|   NULL|    NULL|  NULL|49.325581395348834|20.308604651162792|456.46232558139553|
| stddev|   NULL|    NULL|  NULL|30.078247899067208| 47.34511769375187| 447.0221038416717|
|    min|Central| Andrews|Binder|                 2|              1.29|              9.03|
|    max|   West|Thompson|Pencil|                96|             275.0|           1879.06|
+-------+-------+--------+------+------------------+------------------+------------------+



In [28]:
numeric_columns = [e[0]
         for e in sample_df_inferred.dtypes
         if e[1] in ('int', 'double')
        ]

(
    sample_df_inferred
    .select(numeric_columns)
    .describe()
    .show()
)

+-------+------------------+------------------+------------------+
|summary|             Units|          UnitCost|             Total|
+-------+------------------+------------------+------------------+
|  count|                43|                43|                43|
|   mean|49.325581395348834|20.308604651162792|456.46232558139553|
| stddev|30.078247899067208| 47.34511769375187| 447.0221038416717|
|    min|                 2|              1.29|              9.03|
|    max|                96|             275.0|           1879.06|
+-------+------------------+------------------+------------------+



In [29]:
sample_df_inferred.agg(*
    [f.mean(f.col(e)).alias('mean_' + e) for e in numeric_columns] +
    [f.stddev(f.col(e)).alias('stddev_' + e) for e in numeric_columns]
).show()

+------------------+------------------+------------------+------------------+-----------------+-----------------+
|        mean_Units|     mean_UnitCost|        mean_Total|      stddev_Units|  stddev_UnitCost|     stddev_Total|
+------------------+------------------+------------------+------------------+-----------------+-----------------+
|49.325581395348834|20.308604651162792|456.46232558139553|30.078247899067208|47.34511769375187|447.0221038416717|
+------------------+------------------+------------------+------------------+-----------------+-----------------+



## .distinct()

In [30]:
sample_df_inferred.distinct().show(4)

Py4JJavaError: An error occurred while calling o215.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 1 times, most recent failure: Lost task 0.0 in stage 34.0 (TID 27) (92a28b22707a executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '1/6/16' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48)
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218)
	at org.apache.spark.

In [31]:
(
    sample_df_inferred
    .select('Region', 'Rep')
    .distinct()
    .orderBy('Region', 'Rep')
    .show()
)

+-------+--------+
| Region|     Rep|
+-------+--------+
|Central| Andrews|
|Central|    Gill|
|Central| Jardine|
|Central|  Kivell|
|Central|  Morgan|
|Central|   Smith|
|   East|  Howard|
|   East|   Jones|
|   East|  Parent|
|   West| Sorvino|
|   West|Thompson|
+-------+--------+

